# Solution Revenge: Catching the Mice

  * John Mount
  * Nina Zumel
  * https://www.win-vector.com
  * October 11, 2024

## Introduction

We are back, after being beaten by Dudeney's puzzle solving ability (not by the puzzle!). Let's build a more compact pencil and paper method to enumerate essentially all solutions - no holds barred!

The puzzle is as follows.

<center>
<img src="mouse_puzzle.png">
</center>

And our (legit!) translation of it into equations and legit "could be done by hand' solution is [here](chatching_the_mice_sieve.ipynb). We strongly suggest reading that first.

## Solving the problem with modular equations

Let `advance` be the number of steps forward the cat takes at each step. We count motion from the first mouse after where we start. We are starting at mouse `0`.


To eat the white mouse on the third move, we must not eat it in the first two moves and we must eat the mouse in position 0 at the end of the 3rd move. In our previous note we said this was captured exactly by the following simultaneous nested equations.

  * `(advance % 13) != 0`
  * `((advance % 13) + advance) % 12 != 0`
  * `(((advance % 13) + advance) % 12 + advance) % 11 = 0`.

With some effort (for details, please see appendices) this implies the following non-nested equations must hold:

  * `(advance % 13) != 0`
  * `(advance % 13) + (advance % 12) != 12`
  * `(advance % 13) + (advance % 12) + (advance % 11) = 11 or 23`.

Note: The de-nesting lemma given in an appendix gives us `(advance % 13) + (advance % 12) + (advance % 11) = 0, 11, 12, or 23`. `0` is eliminated *a priori*, as it would imply `(advance % 13) == 0`, which violates the first check equation. `12` is eliminated through a domain specific argument in an appendix.

There are standard methods known in 1908 that can move through groups of solutions to the second non-nested set of equations.

In this note we will call the first two non-nested equations "the basic checks" and concentrate on the third check `(advance % 13) + (advance % 12) + (advance % 11)`.


## Notation relating integers to vectors

In [1]:
# imports
import functools
import textwrap
import inspect
import numpy as np
import sympy as sp
from sympy.ntheory.modular import crt
from IPython.display import Code, display
from catching_the_mice_fns import (
    check_mice_equations,
    check_soln,
    find_candidates_11_12_13,
    run_cat_process,
    WHITE_MOUSE,
)

In [2]:
# write down our moduli
moduli = (11, 12, 13)
# for number theory reasons we consider only solutions with different remainders by
# the shared_modulus as truly different solutions
shared_modulus = functools.reduce(sp.lcm, moduli)
assert shared_modulus == 1716  # 11 * 12 * 13

shared_modulus

1716

For a non-negative integer `x < 1716` we consider `x` to be in one to one correspondence with the integer vector `(x%11, x%12, x%13)`. The reason this is a one to one correspondence, and how to reverse it (move from `(a, b, c)` to a matching `x`) is the content of the [Chinese Remainder Theorem](https://crypto.stanford.edu/pbc/notes/numbertheory/crt.html) or CRT. This method would surely have been known to Dudeney.

Throughout this note we will move from integers to vectors and vectors to integers. We will also exploit that the CRT implied mapping preserves additive structure: ie. `x ~ (a, b, c)` and `y ~ (d, e, f)` means
```
(x + y) % 1716 ~ ((a + d) % 11, (b + e) % 12, (c + f)%13).
```

### Moving from solution to solution 

#### 12 is a magic number!

We show in the appendix that if `advance` is a valid solution then `advance + 12` is also a valid solution---*if* it doesn't violate the first two check equations. That is to say, when we shift by `12` we don't have to check the third equation (though we must ensure we have not violated any of the first two). The intuition is: `(12 - 1) % 11 = 0`, `(12 - 0) % 12 = 0`, and `(12 + 1) % 13 = 0`. Or in vector form `12 ~ (1, 0, -1)`. Adding `12` to a solution represents moving mass from the length `13` congruence and adding it to the length `11` congruence, preserving the net precession.

#### Finding another solution to solution move

We want a non-negative integer `z` that represents a vector move of `(-1, 1, 0)`. The CRT can find `z`. We will call a function, but Dudeney would have computed this using pencil and paper.

In [3]:
# write down our desired pattern: ((z + 1) % 11 == 0, (z - 1) % 12 == 0, z % 13 = 0
# This is a little confusing but (for the first term) you can think of it as:
# "if (z + 1) behaves like 0, then z behaves like -1"
# hence the equation we are solving is "-1 = z % 11"
pattern = (-1, 1, 0)
# ask the CRT to build v_11, v_12, and v_23
delta_m1_p1_0 = int(crt(moduli, pattern)[0])
assert delta_m1_p1_0 == 637  # the value we saw last time we calculated this

delta_m1_p1_0

637

Note that even though we are applying the CRT by calling a `sympy` function, the algorithm is quite [straightforward (if perhaps a little tedious) to execute by hand](https://youtu.be/ru7mWZJlRQg?si=pkHK-1wb9C1nl-Qa), as Dudeney would have done. Now let's confirm that this solution gives us the pattern we want.

In [4]:
# confirm pattern
residuals_from_pattern = (
    (delta_m1_p1_0 + 1) % 11,
    (delta_m1_p1_0 - 1) % 12,
    delta_m1_p1_0 % 13,
)
assert residuals_from_pattern == (0, 0, 0)

residuals_from_pattern

(0, 0, 0)

We prepare a table of known solution to solution moves as follows. 

In [5]:
# encode what we need about moving from solution to solution
d_vectors = {
    (1, 0, -1): 12,
    (-1, 1, 0): delta_m1_p1_0,
}
# add the two together to get a third solution-to-solution move
d_vectors[(0, 1, -1)] = (d_vectors[(1, 0, -1)] + d_vectors[(-1, 1, 0)]) % shared_modulus

d_vectors

{(1, 0, -1): 12, (-1, 1, 0): 637, (0, 1, -1): 649}

We claim:

**If** `x ~ (a, b, c)` is a puzzle solution and `delta` is from our `d_vectors` table such that `delta ~ (d, e, f)`:

  **Then** `(x + delta) % 1716 ~ ((a + d) % 11, (b + e) % 12, (c + f) % 13)` is also a puzzle solution (unless a boundary check fails).

**Because** `(x + delta) % 11 + (x + delta) % 12 + (x + delta) % 13 = a + b + c`


We now, subject to confirmation in the appendix, have systematic ways to move from solution to solution.

### Putting solutions together quickly

It is easy to enumerate all `advance > 0` such that `(advance % 13) + (advance % 12) + (advance % 11)` equal to a given sum. We will use this enumeration to in turn find all solutions to the nested version of the catching the mice problem `((advance % 13) + advance % 12) + advance) % 11` in a pencil and paper friendly manner.

We must arrange the order that we enumerate solutions to respect the first two "easy checks":

  * `(advance + d) % 13 != 0`
  * `((advance + d) % 13 + advance + d) % 12 != 0`

When we ensure this we get the complicated 3rd check for free. Some notes on this are in the appendix. With the assurances from these checks, we can enumerate all solutions.

### Finding all the solutions

Let's find all the solutions.



#### The `x%13 + x%12 + x%11 = 11` case 
For the cases where `x%13 + x%12 + x%11 = 11` we run through solution vectors, of the form `(x%11, x%12, x%13)`

  * We start with a solution `x` of vector form `(0, 0, 11)`. `x = 1584` can be found by the CRT.
  * Run through solutions corresponding to vectors: `(0, 0, 11)`, `(0, 1, 10)`, ..., `(0, 10, 1)` by examining `(x + a * 649) % 1716` (`649` being the value from the `d_vectors` table that moves mass from third vector position to the second vector position).
  * For each such solution `y = (x + a * 649) % 1716` we look at more solutions of the form `(y + b * 12) % 1716` for more solutions. Note: this moves mass into the first position of our vectors.

A pencil and paper solver could quickly write out the following ledger of results. In fact the procedure is easier to execute than to describe.

In [6]:
solns_11 = find_candidates_11_12_13(11, d_vectors=d_vectors)

solutions to ((x%13 + x)%12 + x)%11 = 0 such that (x%13) + (x%12) + (x%11) = 11
vector notation: (soln % 11, soln % 12, soln % 13)
start: (0, 0, 11) -> 1584
  more solution(s) (by +12 rule): 1596, 1608, 1620, 1632, 1644, 1656, 1668, 1680, 1692, 1704
step: (0, 1, 10) -> 1584 + 649 = 2233 =(-1*1716)= 517
  more solution(s) (by +12 rule): 529, 541, 553, 565, 577, 589, 601, 613, 625
step: (0, 2, 9) -> 517 + 649 = 1166
  more solution(s) (by +12 rule): 1178, 1190, 1202, 1214, 1226, 1238, 1250, 1262
step: (0, 3, 8) -> 1166 + 649 = 1815 =(-1*1716)= 99
  more solution(s) (by +12 rule): 111, 123, 135, 147, 159, 171, 183
step: (0, 4, 7) -> 99 + 649 = 748
  more solution(s) (by +12 rule): 760, 772, 784, 796, 808, 820
step: (0, 5, 6) -> 748 + 649 = 1397
  more solution(s) (by +12 rule): 1409, 1421, 1433, 1445, 1457
step: (0, 6, 5) -> 1397 + 649 = 2046 =(-1*1716)= 330
  more solution(s) (by +12 rule): 342, 354, 366, 378
step: (0, 7, 4) -> 330 + 649 = 979
  more solution(s) (by +12 rule): 991, 1003,

#### The `x%13 + x%12 + x%11 = 23` case 

We finish by running inspection process for the case when `x%13 + x%12 + x%11 = 23`, finding the remaining solutions. Again we imagine doing this by hand.

In [7]:
solns_23 = find_candidates_11_12_13(23, d_vectors=d_vectors)

solutions to ((x%13 + x)%12 + x)%11 = 0 such that (x%13) + (x%12) + (x%11) = 23
vector notation: (soln % 11, soln % 12, soln % 13)
start: (10, 1, 12) -> 1429
step: (9, 2, 12) -> 1429 + 637 = 2066 =(-1*1716)= 350
  more solution(s) (by +12 rule): 362
step: (8, 3, 12) -> 350 + 637 = 987
  more solution(s) (by +12 rule): 999, 1011
step: (7, 4, 12) -> 987 + 637 = 1624
  more solution(s) (by +12 rule): 1636, 1648, 1660
step: (6, 5, 12) -> 1624 + 637 = 2261 =(-1*1716)= 545
  more solution(s) (by +12 rule): 557, 569, 581, 593
step: (5, 6, 12) -> 545 + 637 = 1182
  more solution(s) (by +12 rule): 1194, 1206, 1218, 1230, 1242
step: (4, 7, 12) -> 1182 + 637 = 1819 =(-1*1716)= 103
  more solution(s) (by +12 rule): 115, 127, 139, 151, 163, 175
step: (3, 8, 12) -> 103 + 637 = 740
  more solution(s) (by +12 rule): 752, 764, 776, 788, 800, 812, 824
step: (2, 9, 12) -> 740 + 637 = 1377
  more solution(s) (by +12 rule): 1389, 1401, 1413, 1425, 1437, 1449, 1461, 1473
step: (1, 10, 12) -> 1377 + 637 = 20

#### The solutions

The returned solutions are the following integers, which can be mined from the ledger.

In [8]:
all_solutions = solns_11 + solns_23
assert len(all_solutions) == len(set(all_solutions))

print("\n".join(textwrap.wrap(str(all_solutions))))

[1584, 1596, 1608, 1620, 1632, 1644, 1656, 1668, 1680, 1692, 1704,
517, 529, 541, 553, 565, 577, 589, 601, 613, 625, 1166, 1178, 1190,
1202, 1214, 1226, 1238, 1250, 1262, 99, 111, 123, 135, 147, 159, 171,
183, 748, 760, 772, 784, 796, 808, 820, 1397, 1409, 1421, 1433, 1445,
1457, 330, 342, 354, 366, 378, 979, 991, 1003, 1015, 1628, 1640, 1652,
561, 573, 1210, 1429, 350, 362, 987, 999, 1011, 1624, 1636, 1648,
1660, 545, 557, 569, 581, 593, 1182, 1194, 1206, 1218, 1230, 1242,
103, 115, 127, 139, 151, 163, 175, 740, 752, 764, 776, 788, 800, 812,
824, 1377, 1389, 1401, 1413, 1425, 1437, 1449, 1461, 1473, 298, 310,
322, 334, 346, 358, 370, 382, 394, 406, 935, 947, 959, 971, 983, 995,
1007, 1019, 1031, 1043, 1055]


Notice we are not sorting the solutions- as that isn't a pleasant pencil and paper step. We can get away with not sorting, as we won't have duplicate solutions we need to eliminate.

## The official solution

Let's get back to the official solution that made us wonder if Dudeney used a sieve method (as we speculated in our first solution), or did something a bit flashier.

<img src="mouse_answer.png">

We can confirm all claims in the solution writeup.

First the values `100 - 1` and `1000 - 1` (remember we are numbering one less than Dudeney) are indeed solutions that we found.

In [9]:
assert 99 in all_solutions
assert 999 in all_solutions

And, `100 - 1` is indeed the smallest solution.

In [10]:
assert 99 == np.min(all_solutions)

We confirm there are exactly 72 more solutions in the interior of the claimed interval.

In [11]:
n_interior = len([si for si in all_solutions if (si > 99) and (si < 999)])
assert n_interior == 72

n_interior

72

There, we didn't have to sieve through 1000 candidates to confirm Dudeney's result. We can also be overly clever.

## Comments

Puzzle solving is problem solving. With the right puzzles you are able to try methods on essential difficulties, while putting off dealing with many inessential difficulties. A good puzzle can be a rehearsal for future good work.

This problem has more mathematical structure than we expect in magazine puzzles. We believe one big trick is realizing solutions may be a number much larger than 13. 

The "archeology" of how could this have been solved at the time really appealed to us. To us the problem stank of number theory. Our first guess was some sort of [sieve method](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) crossing out non-solutions. This works and can quickly find the smallest solution. However, it seemed unlikely someone would run the Sieve up through 1000 to get the further claims in the answer. After seeing the claim of counting solutions (which is typically hard) we settled on methodologies that step through solutions, or even better step over groups of solutions.

Nina worked through a technique of re-writing the nested equations as case based simultaneous equations. This felt similar to the solutions to [100 Bushels of Corn](https://ninazumel.com/blog/2024-09-26-100bushels/). It feels like the current writeup may be a shorter approach, as it uses the conditions on neighboring solutions to specify cases. John originally found the solution shift `12` from the CRT, and only later noticed that it is "an obvious thing to consider."

There is a risk that CRT was not quite the right tool, leading to larger than desired writeup. However, CRT seems so close to the problem.

A modern reader might also try a dynamic programming table to break down the problem.

## More in this mathematical puzzles series

This puzzle turned out to be a bit harder than we anticipated. More friendly puzzles in this series include:

  * [100 Bushels of Corn](https://ninazumel.com/blog/2024-09-26-100bushels/)
  * [Solving 100 Bushels Using Matrix Factorization](https://win-vector.com/2024/09/29/solving-100-bushels-using-matrix-factorization/)
  * [Bachet’s Four Weights Problem](https://ninazumel.com/blog/2024-09-29-four-weights/)
  * [The Perplexed Banker](https://ninazumel.com/blog/2024-10-03-perplexed-banker/)
  * [Dudeney’s Remainder Problem](https://win-vector.com/2024/10/06/dudeneys-remainder-problem/)
  * [Coin Puzzles](https://ninazumel.com/blog/2024-10-08-coin-puzzles/)
  * [The Wine Thief Problem](https://ninazumel.com/blog/2024-10-10-wine-thief/)

## Appendices

### Brute force 

As we *do* have a computer handy, we might as well confirm all of our calculations by brute force. This is the solution strategy engineers would pursue- run the actual procedure for many different proposed d_vectors (perhaps using markers or tokens). It is also an excellent method, as it requires little code, few tools, few assumptions, and is easy to check. It just wouldn't be available in 1908 for a non-valuable problem.

In [12]:
# brute force check all solutions and non-solutions
for advance in all_solutions:
    assert check_soln(run_cat_process(start=0, advance=advance, k=3))
for advance in set(range(shared_modulus + 1)) - set(all_solutions):
    assert not check_soln(run_cat_process(start=0, advance=advance, k=3))
for advance in range(0, shared_modulus + 2):
    is_soln = check_soln(run_cat_process(start=0, advance=advance, k=3))
    check_1 = check_mice_equations(advance)
    assert is_soln == check_1

### Code

The Python functions used are [here](catching_the_mice_fns.py).

## Confirming the check equations

### The de-nesting lemma


Lemma 1: De-Nesting Lemma

<pre>
  Take integer <code>a > 1</code> and non-negative integers <code>c</code>, <code>x</code>, <code>y</code> with <code>c &lt; a</code>.
  
  Define:

    <code>N = (x % (a + 1) + y) % a</code>
    <code>S = (x % (a + 1)) + (y % a)</code>


  Then <code>N = c</code> <em>if and only if</em> <code>S = c or c + a</code>.
</pre>

Proof:

<pre>
  Consider <code>S - N</code>. Proving the following three claims implies the result.
  To do this we need to perform some simplifications the are only true inside a "modulo a" context
  and some different simplifications that are true outside that context.

  1) Show <code>S - N &ge; 0</code>:
       <code>S - N
          = (x % (a + 1)) + (y % a) - (x % (a + 1) + y) % a
          &ge; ((x % (a + 1)) + (y % a)) % a - (x % (a + 1) + y) % a
          = (x % (a + 1) + y) % a - (x % (a + 1) + y) % a
          = 0</code>
  2) Show <code>S - N &le; a</code>:
       <code>S - N
          = (x % (a + 1)) + (y % a) - (x % (a + 1) + y) % a
          &le; (x % (a + 1)) + (y % a) - (x % (a + 1)) % a - (y % a)
          = (x % (a + 1)) - (x % (a + 1)) % a
          = 0 or a</code>   (Note: we have not <em>yet</em> established <code>S - N = 0 or a</code> due to the <code>&le;</code> step.)
  3) Show <code>(S - N) % a = 0</code>:
       <code>(S - N) % a
          = ((x % (a + 1)) + (y % a) - (x % (a + 1) + y) % a) % a
          = ((x % (a + 1)) + (y % a) - x % (a + 1) - (y % a)) % a
          = 0 % a
          = 0</code>

  QED
</pre>

### Applying the de-nesting lemma

We want to relate `(((d % 13) + d) % 12 + d) % 11` to `(d % 13) + (d % 12) + (d % 11)`.

Write:

  * `N = ((d % 13) + d) % 12` by the de-nesting lemma we know that `(d % 13) + (d % 12) = N or N + 12`.
  * `M = ((N % 12) + d) % 11` by the de-nesting lemma we know that `(N % 12) + (d % 11) = M or M + 11`.
  * Then:
    * `(((d % 13) + d) % 12 + d) % 11`
    * `  = ((N % 12) + d) % 11`
    * `  = (N % 12) + (d % 11) - (0 or 11)`
    * `  = N + (d % 11) - (0 or 11)`
    * `  = ((d % 13) + d) % 12 + (d % 11) - 0 or 11`
    * `  = (d % 13) + (d % 12) - (0 or 12) + (d % 11) - (0 or 11)`
    * `  = (d % 13) + (d % 12) + (d % 11) - (0, 11, 12, or 23)`

This give us the desired `(((d % 13) + d) % 12 + d) % 11 = 0` implies `(d % 13) + (d % 12) + (d % 11) = 0, 11, 12, or 23`.

If `(d % 13) + (d % 12) + (d % 11) = 0` then `(d % 13) = 0` (as all the terms in the sum are non-negative). `(d % 13) = 0` is prohibited in our application, allowing us to ignore that case.

There also are no solutions to the mice problem with `(d % 13) + (d % 12) + (d % 11) = 12`. And the example `d = 1453` demostrates this.

#### The `1453` counter example

`d = 1453` shows that the `(d % 13) + (d % 12) + (d % 11) = 12` candidates include non-solutions. We show this by a failing example here. We show there are in fact no solutions with `(d % 13) + (d % 12) + (d % 11) = 12` in a later appendix.

In [13]:
# linearized checks
lc_1453 = (
    (1453 % 13),  # not zero, as we want
    (1453 % 13) + (1453 % 12),  # not zero, as we want
    (1453 % 13) + (1453 % 12) + (1453 % 11),  # 12, which turns out to not imply 0 on the nested checks
)
assert lc_1453[0] != 0
assert lc_1453[1] != 0
assert lc_1453[2] == 12

lc_1453

(10, 11, 12)

In [14]:
# confirm non-solution
traj_1453 = run_cat_process(start=0, advance=1453, k=3)
assert not check_mice_equations(1453)

traj_1453

('black mouse 10', 'black mouse 12', 'black mouse 1')

### Confirming the shift claims

For all shift claims we going to assume both `d` and `d + shift` obey all the basic check conditions (prior to the 3rd check condition):

  * `d > 0`
  * `(d % 13) != 0`
  * `((d % 13) + d) % 12 != 0`
  *  `(d + shift) > 0`
  * `((d + shift) % 13) != 0`
  * `(((d + shift) % 13) + (d + shift)) % 12 != 0`

And that `d` is a valid solution:

  * `(((d % 13) + d) % 12 + d) = 0`

In all cases we wil show all of the above can be combined to show:

 * `((((d + shift) % 13) + (d + shift)) % 12 + (d + shift)) = 0`

I.e.: `d + shift` is also a valid solution. We can assume `shift >= 0` and `shift` isn't too large, as a negative shift can be re-mapped to `shift % 1716`.
 

#### Confirming the shift by `12` claim

Given the above assumptions we will show `d + 12` is a valid solution.

<pre>
<code>(((d + 12)%13 + (d + 12))%12 + (d + 12))%11</code>
<code>  = (((d - 1)%13 + d)%12 + d + 1)%11</code>
<code>  = ((d%13 - 1 + d)%12 + d + 1)%11</code> (valid because by check conditions <code>d > 0</code> and <code>d % 13 > 0</code>)
<code>  = ((d%13 + d)%12 - 1 + d + 1)%11</code> (valid because by check condition <code>((d % 13) + d) % 12 > 0</code>)
<code>  = ((d%13 + d)%12 + d)%11</code>
<code>  = 0</code> (as `d` already meets all our check conditions)
</pre>

Note: as we have all the basic checks for `d` and `d + 12` in our assumptions. We can reverse this argument and show `d + 12` a solution implies `d` is a solution (when the basic checks hold).

#### Confirming the shift by `637` claim

Given the above assumptions we will show `d + 637` is a valid solution.

<pre>
<code>(((d + 637)%13 + (d + 637))%12 + (d + 637))%11</code>
<code>  = ((d%13 + d + 1)%12 + d - 1)%11</code>
<code>  = ((d%13 + d)%12 + 1 + d - 1)%11</code> (requires <code>(d%13 + d)%12 != 11</code>, implied by our conditions).
<code>  = ((d%13 + d)%12 + d)%11</code>
<code>  = 0</code> (as <code>d</code> already meets all our check conditions)
</pre>

Note: as we have all the basic checks for `d` and `d + 12` in our assumptions. We can reverse this argument and show `d + 637` a solution implies `d` is a solution (when the basic checks hold).

#### Confirming the shift by `649` claim

This is as transitive implication of the previous two claims, as we can write `649 = 12 + 637`. This is using the fact that we have the same linear structure in solution integers and solution vectors.

### Eliminating the `x%13 + x%12 + x%11 = 12` case 

When `x%13 + x%12 + x%11 = 12`, there are no solutions. 

We can establish this quickly.

Consider all `x` such that `x` is a solution to the catching mice problem with `x%13 + x%12 + x%11 = 12`. First we show `x%11 < 10` for any such solution. By our other conditions, the only possible only solutions with `x%11 >= 10` have `(x%11, x%12, x%13) = (10, 0, 2) or (10, 1, 1)`. It turns out neither of these is a valid solution as we show here.


In [15]:
traj_912 = run_cat_process(start=0, advance=int(crt(moduli, (10, 0, 2))[0]), k=3)
assert not check_soln(traj_912)

traj_912

('black mouse 2', 'black mouse 3', 'black mouse 1')

In [16]:
traj_1561 = run_cat_process(start=0, advance=int(crt(moduli, (10, 1, 1))[0]), k=3)
assert not check_soln(traj_1561)

traj_1561

('black mouse 1', 'black mouse 3', 'black mouse 2')

Suppose `x` is a solution with `x%13 + x%12 + x%11 = 12` and `x%11` maximal. This would imply that at least one of `x -> (x + 12) % 1716` or `x -> (x - 637) % 1716` is a valid solution to valid solution move stealing mass from one of `x%12` or `x%13` and adding it to `x%11`. As `x%11 <= 9` we know that we have room to add more mass to `x%11`. We also know either `x%12 > 0` or `x%13 > 1`, so at least one of them has mass to spare without violating any of the basic check equations. However applying either move increases `x%11`, contradicting the supposed maximality. Therefore there are no solutions with `x%13 + x%12 + x%11 = 12`.
